# Read TWDB water use excel files and prepare them for WaDE tables

# WaDE 
http://wade.westernstateswater.org/

In [2]:
# Import libraries
import xlsxwriter
import xlrd as excel
from openpyxl import load_workbook
import os
import datetime
import pandas as pd

print 'The Python 2.7 libraries are imported'

The Python 2.7 libraries are imported


# Read the input files, prepare it, then export them to excel 

# Map the TWDB headers to WaDE excel headers

ORGANIZATION_ID  
REPORT_ID  
REPORT_UNIT_ID  
BENEFICIAL_USE_ID  
SUMMARY_SEQ  
ROW_SEQ  
AMOUNT  
CONSUMPTIVE_INDICATOR  
METHOD_ID  
START_DATE  
END_DATE  




### First add some code to check in the type of the input file 

If the excel file name contains "Basin" then AreaType="Basin"

elif the excel file name contains "County" then AreaType="County"

else print 'The file name is not supported'

We will use this "AreaType" below to look up the identifier of the basin or county inside each file


### Read the excel file County_Basin_IDs.xlsx and its two sheets: County_FIPS, BasinTCEQID
### so basically I want to print a new column to the result sheet for the identifier of each "name" in the input data
In the County_FIPS sheet, store the two columns "COUNTYNAME" and "COUNTYFP" together which will use them in a look up function

The look up works like this:
We have a column called County in the input excel files, for each row, we need to look up the COUNTYFP for it from the above pair.
Where 
County=COUNTYNAME  
Get COUNTYFP

In the BasinTCEQID sheet, store the two columns "BasinName" and "BasinID" together which will use them in a look up function

The look up works like this:
We have a column called "Basin" in the input excel files, for each row, we need to look up the Name for it from the above pair.
where Name=BasinName  
Get BasinID


### add a new column called "ReportingUnitID" which takes the value of the  COUNTYFP or the BasinID


## Add a new sheet to the same exported file
call it 

SUMMARY_USE

first print these headers to it

ORGANIZATION_ID,REPORT_ID,REPORTING_UNIT_ID,BENEFICIAL_USE_ID,SUMMARY_SEQ,FRESH_SALINE_IND,
SOURCE_TYPE,POWER_GENERATED,POPULATION_SERVED,WFS_FEATURE_REF

ORGANIZATION_ID='TWDB'  
REPORT_ID=fill in the year value  
REPORTING_UNIT_ID=fill in the County or Basin ID value  
BENEFICIAL_USE_ID='6' # we need to look it up later  # yes it would take the value from LU_SEQ_NO

SUMMARY_SEQ='1'  
FRESH_SALINE_IND='1'  
SOURCE_TYPE='1'  
POWER_GENERATED=''  

if attribute contains "Municipal" then  
POPULATION_SERVED=Population   

else  
POPULATION_SERVED=""  

WFS_FEATURE_REF=""  


## the number of rows in the oputput in this sheet are the same like the input file (23) or 400 rows depending on the source file.


# Add a new sheet called REPORTING_UNIT  

ORGANIZATION_ID,REPORT_ID,REPORTING_UNIT_ID,REPORTING_UNIT_NAME,REPORTING_UNIT_TYPE,STATE,
COUNTY_FIPS,HUC   


WHERE   

ORGANIZATION_ID='TWDB'     
REPORT_ID=fill in the year value      
REPORTING_UNIT_ID=fill in the County or Basin ID value      
REPORTING_UNIT_NAME= fill in the name  
REPORTING_UNIT_TYPE=fill in the ReportUnitType   

STATE='55'  

if REPORTING_UNIT_TYPE=="County"  
Then COUNTY_FIPS= REPORTING_UNIT_ID  
else COUNTY_FIPS= ""

HUC=""  



## the # of rows here is similair to the source file (23 or 400) depending on the file type  


## Add a new sheet called REPORT  
add the headers to it
ORGANIZATION_ID,REPORT_ID,REPORTING_DATE,REPORTING_YEAR,REPORT_NAME,REPORT_LINK,YEAR_TYPE


WHERE   

ORGANIZATION_ID='TWDB'       
REPORT_ID=fill in the year value from the folder name e.g., "2015"    
REPORTING_DATE= get the date of today like this format 8/15/2018  

REPORT_NAME= get the excel file name  

REPORT_LINK="http://www.twdb.texas.gov/waterplanning/waterusesurvey/estimates/index.asp"
 

YEAR_TYPE="Calendar year"


## the # of rows here is like the number of excel files (each excel file has one row stored)




# Read the excel file County_Basin_IDs.xlsx 

Get the new sheet called: LU_BENEFICIAL_USE  

In the LU_BENEFICIAL_USE sheet, store the two columns "Attribute" and "LU_SEQ_NO"  
which will use them in a look up function  

The look up works like this:  
We have a column called Attribute in the input excel files, for each row, we need to look up the "LU_SEQ_NO" for it from the above pair.   

Where    
Attribute==Attribute    
Get LU_SEQ_NO   

then set the BENEFICIAL_USE_ID=LU_SEQ_NO  

then use this value in S_USE_AMOUNT table  

## Update these columns to the S_USE_AMOUNT sheet 

ORGANIZATION_ID,REPORT_ID,REPORTING_UNIT_ID,BENEFICIAL_USE_ID,SUMMARY_SEQ,ROW_SEQ,AMOUNT,
CONSUMPTIVE_INDICATOR,METHOD_ID,START_DATE,END_DATE

WHERE 

ORGANIZATION_ID='TWDB'    
REPORT_ID=fill in the year value    
REPORTING_UNIT_ID=fill in the County or Basin ID value    

BENEFICIAL_USE_ID= use the look up SEQ_NO value   
SUMMARY_SEQ='1'    
ROW_SEQ='1'  
AMOUNT=Value

CONSUMPTIVE_INDICATOR="2"  
METHOD_ID="1"  
START_DATE="01/01"  
END_DATE="12/31"  


# add another sheet called "S_USE_IRRIGATION"

first print these headers to the sheet  

ORGANIZATION_ID,REPORT_ID,REPORTING_UNIT_ID,BENEFICIAL_USE_ID,SUMMARY_SEQ,IRRIGATION_SEQ, IRRIGATION_METHOD, ACRES_IRRIGATED,CROP_TYPE  

this sheet uses the same data for the first five columns like the first sheet called "S_USE_AMOUNT" only if the next condition is met  
 
if attribute contains "Irrigation" then  

ORGANIZATION_ID='TWDB'  
REPORT_ID=fill in the year value  
REPORTING_UNIT_ID=fill in the County or Basin ID value  
BENEFICIAL_USE_ID= use the look it SEQ_NO
SUMMARY_SEQ='1'  


IRRIGATION_SEQ='1'  
IRRIGATION_METHOD=""  
ACRES_IRRIGATED=''  
CROP_TYPE=""  






In [4]:
# open sheet to write new xlsx

# output file name
full_path = 'TWDB_WaDE_ReadyInput.xlsx'
workbook1 = xlsxwriter.Workbook(full_path)

cell_format_field_name = workbook1.add_format()
cell_format_field_name.set_font_size(14)
cell_format_field_name.set_bold()
print full_path

# Read all the excel files inside the /InputFiles/ folder
import os
for filename in os.walk("../InputFiles"):
    for f in filename[0]:
        print f
#     if os.path.isdir(os.path.join(os.path.abspath("../InputFiles"), filename)):
#         print os.path.join(os.path.abspath("../InputFiles"), filename)
#         for inputfilename in os.listdir(os.path.join(os.path.abspath("."), filename)):
#             print inputfilename

TWDB_WaDE_ReadyInput.xlsx


# Read sheets from the TWDB



save sheets to this dir
print os.path.abspath("./InputFiles/CSV_ReadyWaDE_Input")


In [5]:
# open sheet to write new xlsx

# get BENEFICIAL_USE_ID
book_to_get_BENEFICIAL_USE_ID = excel.open_workbook('County_Basin_IDs.xlsx')
sheet_to_get_BENEFICIAL_USE_ID = book_to_get_BENEFICIAL_USE_ID.sheet_by_name('LU_BENEFICIAL_USE')
rows = [rows for rows in sheet_to_get_BENEFICIAL_USE_ID.get_rows()]
temp_data = rows[1:][:]
BENEFICIAL_USE_ID_dic_data = []
for i, dd in enumerate(temp_data):
    id_val = str(int(dd[0].value))
    attr_val = dd[4].value
#     print id_val
#     print attr_val
    BENEFICIAL_USE_ID_dic_data.append([attr_val.replace(' ', '').lower(), id_val])
    
sheet_to_get_BASIN_TCEQID = book_to_get_BENEFICIAL_USE_ID.sheet_by_name('Basin_TCEQID')
rows1 = [rows for rows in sheet_to_get_BASIN_TCEQID.get_rows()]
temp_data = rows1[1:][:]
Basin_TCEQID_dictData = {}
for i, dd in enumerate(temp_data):
#     print dd[2].value
    id_val = str(int(dd[1].value))
    attr_val = dd[2].value
#     print id_val
#     print attr_val
    Basin_TCEQID_dictData[attr_val.replace(' ', '').lower()] = id_val
    
sheet_to_get_County_Fips = book_to_get_BENEFICIAL_USE_ID.sheet_by_name('County_FIPS')
rows = [rows for rows in sheet_to_get_County_Fips.get_rows()]
temp_data = rows[1:][:]
County_FIPS_dictData = {}
for i, dd in enumerate(temp_data):
    id_val = str(int(dd[2].value))
    attr_val = dd[3].value
#     print id_val
#     print attr_val
    County_FIPS_dictData[attr_val.replace(' ', '').lower()] = id_val

    

# output file name
file_path_to_write = 'TWDB_WaDE_ReadyInput.xlsx'


workbook1 = xlsxwriter.Workbook(file_path_to_write)

cell_format_field_name = workbook1.add_format()
cell_format_field_name.set_font_size(14)
cell_format_field_name.set_bold()

workbook1.add_worksheet("S_USE_AMOUNT")
workbook1.add_worksheet("SUMMARY_USE")
workbook1.add_worksheet("REPORT")
workbook1.add_worksheet("REPORTING_UNIT")
workbook1.add_worksheet("S_USE_IRRIGATION")



workbook1.close()

data_to_write_in_report = []
data_to_write_in_Summary = []
data_to_write_in_reporting_unit = []
data_to_write_irrigation = []
data_to_write_amount = []


workbook = load_workbook(file_path_to_write)
sheet_Amount = workbook.get_sheet_by_name("S_USE_AMOUNT")
sheet_SUMMARY_USE = workbook.get_sheet_by_name("SUMMARY_USE")
sheet_reporting_unit = workbook.get_sheet_by_name("REPORTING_UNIT")
sheet_report = workbook.get_sheet_by_name("REPORT")
sheet_irrigation = workbook.get_sheet_by_name('S_USE_IRRIGATION')

# add fields names
field_names_Amount = ["ORGANIZATION_ID","REPORT_ID","REPORTING_UNIT_ID","BENEFICIAL_USE_ID","SUMMARY_SEQ","ROW_SEQ","AMOUNT","CONSUMPTIVE_INDICATOR","METHOD_ID","START_DATE","END_DATE"]
for col_id, field_name in enumerate(field_names_Amount):
    sheet_Amount.cell(row=1, column=col_id + 1, value=field_name)
data_to_write_amount.append(field_names_Amount)
    
field_names_Summary = ["ORGANIZATION_ID", "REPORT_ID", "REPORTING_UNIT_ID", "SUMMARY_SEQ","BENEFICIAL_USE_ID",   "FRESH_SALINE_IND", "SOURCE_TYPE", "POWER_GENERATED", "POPULATION_SERVED", "WFS_FEATURE_REF"]
for col_id, field_name in enumerate(field_names_Summary):
    sheet_SUMMARY_USE.cell(row=1, column=col_id + 1, value=field_name)
data_to_write_in_Summary.append(field_names_Summary)
    
field_names_reporting_unit = ["ORGANIZATION_ID","REPORT_ID","REPORTING_UNIT_ID","REPORTING_UNIT_NAME","REPORTING_UNIT_TYPE","STATE","COUNTY_FIPS","HUC"]
for col_id, field_name in enumerate(field_names_reporting_unit):
    sheet_reporting_unit.cell(row=1, column=col_id + 1, value=field_name)
data_to_write_in_reporting_unit.append(field_names_reporting_unit)

field_names_Report = ["ORGANIZATION_ID","REPORT_ID","REPORTING_DATE","REPORTING_YEAR","REPORT_NAME","REPORT_LINK","YEAR_TYPE"]
for col_id, field_name in enumerate(field_names_Report):
    sheet_report.cell(row=1, column=col_id + 1, value=field_name)
data_to_write_in_report.append(field_names_Report)
    
field_names_irrigation = ["ORGANIZATION_ID","REPORT_ID","REPORTING_UNIT_ID","BENEFICIAL_USE_ID","SUMMARY_SEQ","IRRIGATION_SEQ","IRRIGATION_METHOD","ACRES_IRRIGATED","CROP_TYPE"]
for col_id, field_name in enumerate(field_names_irrigation):
    sheet_irrigation.cell(row=1, column=col_id + 1, value=field_name)
data_to_write_irrigation.append(field_names_irrigation)

start_row_id = 2
start_row_id_for_report = 2
start_row_id_for_irrigation = 2

print 'done'


done


C:\Users\Adel\Anaconda2\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\Adel\Anaconda2\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\Adel\Anaconda2\lib\site-packages\ipykernel_launcher.py:71: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\Adel\Anaconda2\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\Users\Adel\Anaconda2\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


# Prepare data and map it to WaDE sheets

In [6]:
print "walk:"
print os.path.abspath("./InputFiles")
inputfile_pth = os.path.abspath("./InputFiles")
for x in os.walk(inputfile_pth):
    print "walk:" + str(len(x[1]))
    for sub_dir in x[1]:
        file_dir = '{}/{}'.format(inputfile_pth, sub_dir)
        print "sub dir:  "+ file_dir
        for xx in os.walk(file_dir):
            
            for filename in xx[2]:
                print filename
                input_file_full_path = '{}/{}'.format(file_dir, filename)
                print "input_file_full_path: " + input_file_full_path
                
                # full_path = 'TWDB_WaDE_ReadyInput.xlsx'
                

                ##########################################################

                book = excel.open_workbook(input_file_full_path)
                # //////////////////////////////////////////////////////////////////////////////////////////////
                # verify that the input file is a TWDB file
                try:
                    input_sheet = book.sheet_by_index(0)
                except:
                    raise Exception('Input sheet {} not found in the provided Excel file \n Please select valid excel file'.format('SumFinal_BasinReportWithReuse'))
                # # //////////////////////////////////////////////////////////////////////////////////////////////


                # get total cells of input sheet.
                rows = [rows for rows in input_sheet.get_rows()]
                if not rows:
                    print "no rows"
                    continue

                # get headers
                headers = rows[1][9:]
                if not headers:
                    print "no headers"
                    continue

                # get values
                data = [row[0:] for row in rows[2:]]
                if not data:
                    print "no values"
                    continue

                

                area_type = "County"
                if "Basin" in filename:
                    area_type = "Basin"

                # ready data to write
                # First, loop through the headers
                # ORGANIZATION_ID,REPORT_ID,REPORTING_UNIT_ID,BENEFICIAL_USE_ID,SUMMARY_SEQ,IRRIGATION_SEQ, IRRIGATION_METHOD, 
                # ACRES_IRRIGATED,	CROP_TYPE  

                # the data starts in columns after column# 9 
                print "headers:" + str(len(headers))
                
                
                isWroteReportingUnit = False
                
                for i in range(len(headers)):
                    for j, d in enumerate(data):
                        if str(d[1].value.replace(' ', '').lower()) == 'statetotal':
                            continue
                        BENEFICIAL_USE_ID = ''
                        for temp in BENEFICIAL_USE_ID_dic_data:
                            if temp[0] == headers[i].value.replace(' ', '').lower():
                                BENEFICIAL_USE_ID = temp[1]
                    
#                         if not isWroteReportingUnit:
#                             COUNTY_FIPS = ""
#                             if area_type == "County":
#                                 COUNTY_FIPS = str(j)
                            
                            
                        if area_type == "County":
                            ddd = County_FIPS_dictData[str(d[1].value.replace(' ', '').lower())]
                        elif area_type == "Basin":                             
                            ddd = Basin_TCEQID_dictData[str(d[1].value.replace(' ', '').lower())]
                        
                        
                        
                        data_to_write_temp = ['TWDB',str(int(d[0].value)), ddd, BENEFICIAL_USE_ID, '1', '1', str(d[i + 9].value), 'N', '1', '01/01', '12/31']
                        data_to_write_amount.append(data_to_write_temp)
                        
                        
                        if 'irrigation' in headers[i].value.replace(' ','').lower():
                            data_to_write_temp = ['TWDB',str(int(d[0].value)), ddd, BENEFICIAL_USE_ID, '1', '1','','','']
                            data_to_write_irrigation.append(data_to_write_temp)
                        
                        POPULATION_SERVED = ''
                        if 'Municipal' in headers[i].value:
                            POPULATION_SERVED = str(int(d[2].value))
                        
                        #                              Organization,Year,
                        data_to_write_temp_in_Summary = ["TWDB", str(int(d[0].value)), ddd, '1',BENEFICIAL_USE_ID,  '1','1', '', POPULATION_SERVED, '']
                        data_to_write_in_Summary.append(data_to_write_temp_in_Summary)
                        
                        if not isWroteReportingUnit:
                            COUNTY_FIPS = ""
                            if area_type == "County":
                                COUNTY_FIPS = County_FIPS_dictData[str(d[1].value.replace(' ', '').lower())]
                            elif area_type == "Basin":
                                COUNTY_FIPS =""
                            if area_type == "County":
                                ddd = County_FIPS_dictData[str(d[1].value.replace(' ', '').lower())]
                            elif area_type == "Basin":
                                ddd = Basin_TCEQID_dictData[str(d[1].value.replace(' ', '').lower())]
                            
#                             so skip the value the the row called "StateTotal"?
                            
                                
                                                            #Organization,Year,ReportingUnitID,BasinName,
                            data_to_write_temp_in_Summary = ["TWDB", str(int(d[0].value)), ddd, str(d[1].value), area_type, '55',COUNTY_FIPS, '']
                            data_to_write_in_reporting_unit.append(data_to_write_temp_in_Summary)
                    isWroteReportingUnit = True

            data_to_write_temp_in_Summary = ["TWDB", sub_dir, str(datetime.datetime.today().strftime('%m/%d/%Y')),sub_dir ,"", "http://www.twdb.texas.gov/waterplanning/waterusesurvey/estimates/index.asp", 'Calendar Year']
            data_to_write_in_report.append(data_to_write_temp_in_Summary)

                # the data starts in row after row #2 


print 'done'


walk:
C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles
walk:17
sub dir:  C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2000
SumFinal_BasinReport.xlsx
input_file_full_path: C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2000/SumFinal_BasinReport.xlsx
headers:12
SumFinal_CountyReport.xlsx
input_file_full_path: C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2000/SumFinal_CountyReport.xlsx
headers:12
sub dir:  C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2001
SumFinal_BasinReport.xlsx
input_file_full_path: C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2001/SumFinal_BasinReport.xlsx
headers:12
SumFinal_CountyReport.xlsx
input_file_full_path: C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2001/SumFinal_CountyReport.xlsx
headers:12
sub dir:  C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2002
SumFinal_BasinReport.xlsx
input_file_full_path: C:\Users\Adel\Documents\GitHub\TWDB_script\InputFiles/2002/SumFinal_BasinReport.xlsx
headers:12
S

# Write to excle and csv

In [7]:


#write to excel
csv_output_path = os.path.abspath("./CSV_ReadyWaDE_Input")

for rowID, row in enumerate(data_to_write_amount):
    for colID, cell in enumerate(row):
        try:
            sheet_Amount.cell(row=rowID + start_row_id, column=colID + 1, value=unicode(cell))
        except Exception as e:
            raise Exception(e)
df = pd.DataFrame(data_to_write_amount)
df.to_csv('{}/{}.csv'.format(csv_output_path,'S_USE_AMOUNT'), sep=',', encoding='utf-8', index=False, header=False)

print 'Done with Amount sheet'


for rowID, row in enumerate(data_to_write_in_Summary):
    for colID, cell in enumerate(row):
        try:
            sheet_SUMMARY_USE.cell(row=rowID + start_row_id, column=colID + 1, value=unicode(cell))
        except Exception as e:
            raise Exception(e)
df = pd.DataFrame(data_to_write_in_Summary)
df.to_csv('{}/{}.csv'.format(csv_output_path,'SUMMARY_USE'), sep=',', encoding='utf-8', index=False, header=False)



print 'Done with Summary sheet'

for rowID, row in enumerate(data_to_write_in_reporting_unit):
    for colID, cell in enumerate(row):
        try:
            sheet_reporting_unit.cell(row=rowID + start_row_id, column=colID + 1, value=unicode(cell))
        except Exception as e:
            raise Exception(e)
df = pd.DataFrame(data_to_write_in_reporting_unit)
df.to_csv('{}/{}.csv'.format(csv_output_path,'REPORTING_UNIT'), sep=',', encoding='utf-8', index=False, header=False)


print 'Done with REPORTING_UNIT sheet'

for rowID_for_report, row in enumerate(data_to_write_in_report):
    for colID, cell in enumerate(row):
        try:
            sheet_report.cell(row=rowID_for_report + start_row_id_for_report, column=colID + 1, value=unicode(cell))
        except Exception as e:
            raise Exception(e)
df = pd.DataFrame(data_to_write_in_report)
df.to_csv('{}/{}.csv'.format(csv_output_path,'REPORT'), sep=',', encoding='utf-8', index=False, header=False)

for rowID_for_irrigation, row in enumerate(data_to_write_irrigation):
    for colID, cell in enumerate(row):
        try:
            sheet_irrigation.cell(row=rowID_for_irrigation + start_row_id_for_irrigation, column=colID + 1, value=unicode(cell))
        except Exception as e:
            raise Exception(e)
df = pd.DataFrame(data_to_write_irrigation)
df.to_csv('{}/{}.csv'.format(csv_output_path,'S_USE_IRRIGATION'), sep=',', encoding='utf-8', index=False, header=False)
                
#                 start_row_id_for_report += rowID_for_report + 1
#                 start_row_id_for_irrigation += rowID_for_irrigation + 1
#                 start_row_id += rowID + 1

workbook.save(file_path_to_write)

print 'done'

Done with Amount sheet
Done with Summary sheet
Done with REPORTING_UNIT sheet
done
